In [42]:
!pip -q install openai yfinance

In [43]:
import yfinance as yf
import openai
import json

In [44]:
def get_current_stock_price(symbol):
    """Current stock price. NOTE: This is not live stock update. This is delayed by 15 min. """
    stock = yf.Ticker(symbol)
    todays_data = stock.history(period='1d')
    return todays_data['Close'][0]

In [45]:
def summarize_stock_price(message, function_name, function_response, stock):
    """Take the orginal question plus stock price and let GTP summarize it into a natural lanauge."""
    summary = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-0613",
            messages=[
                {"role": "user", "content": f"What is the current stock price for {stock}?"},
                message,
                {
                    "role": "function",
                    "name": function_name,
                    "content": function_response,
                },
            ],
        )
    return summary["choices"][0]["message"]

In [46]:
symbol = "TSLA"

def run_conversation():
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=[{"role": "user", "content": f"What is the current stock price for symbol {symbol}?"}],
        functions=[
            {
                "name": "get_current_stock_price",
                "description": "Get the current stock price for a symbol",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "symbol": {
                            "type": "string",
                            "description": "The stock symbol or code, e.g. APPL, TSLA",
                        }
                    },
                    "required": ["symbol"],
                },
            }
        ],
        function_call="auto",
    )

    message = response["choices"][0]["message"]

    # Check if GPT response has funtion call. 
    # If so, execute the funtion
    if message.get("function_call"):
        function_name = message["function_call"]["name"]

        if function_name == "get_current_stock_price":
            arguments = message["function_call"]["arguments"]
            arguments_json = json.loads(arguments)

            function_response = get_current_stock_price(
                symbol=arguments_json.get("symbol")
            )
            stock_message = f"Price of stock is {function_response}. Note: This is not live. This is 15-min delayed stock price."

            summary = summarize_stock_price(message, function_name, stock_message, symbol)

            return summary

print(run_conversation())

{
  "role": "assistant",
  "content": "The current stock price for Tesla (TSLA) is $256.79. Please note that stock prices are subject to change and may vary depending on the source you use for real-time pricing."
}
